In [1]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import math
import sys
import copy
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore') 
from zipfile import ZipFile
import cv2


In [2]:
# from skimage.transform import rescale, resize, downscale_local_mean
# image = cv2.imread('image.jpg')
# print(image.shape)
# image = resize(image, (400,400),  anti_aliasing=True)
# vis =np.zeros([image.shape[0]  ,image.shape[1]])
# image[:,:,1:3] = np.zeros(image[:,:,2:3] .shape)
# # print(image.shape)


# def bfs(x , y ,c, index):
#     q =[]
#     ans=0

#     q.append((x,y))
#     while len(q)>0:
#         top = q.pop(0)
#         x = top[0]
#         y = top[1]
        
#         ans+=1
#         if ans<100:
#             print(q)
        
#         if x<=0 or y<=0 or x>=image.shape[0]-1 or y>=image.shape[1]-1:
#             vis[x][y]=1
#             continue
#         if vis[x][y]==1:
#             continue
#         if image[x][y][index]-c<=-100 or image[x][y][index]-c>=100:
#             continue
#         vis[x][y]=1
#         image[x][y][index]=c
        
#         q.append((x+1,y))
#         q.append((x-1,y))
#         q.append((x,y+1))
#         q.append((x,y-1))

# bfs(100,100,image[1][1][0],0)
# print(ans)
        
# cv2.imshow('image' , image)

# cv2.waitKey(0)     
        
        


In [7]:
cap  = cv2.VideoCapture(0)
cap.set(3,500)
cap.set(4,500)

cv2.namedWindow('HSV')
cv2.resizeWindow('HSV'  ,650, 250)
def empty(a):
    pass

cv2.createTrackbar('HUE min' , 'HSV' , 0,179 ,empty)
cv2.createTrackbar('HUE max', 'HSV' , 179,179, empty)
cv2.createTrackbar('SAT min', 'HSV' , 0,255 ,empty)
cv2.createTrackbar('SAT max', 'HSV' , 255,255 ,empty)
cv2.createTrackbar('VAL min' , 'HSV', 0,255 ,empty)
cv2.createTrackbar('VAL max' , 'HSV', 255,255 ,empty)

while True:
    ret ,frame = cap.read()
    imghsv= cv2.cvtColor(frame ,cv2.COLOR_BGR2HSV)
    
    h_min = cv2.getTrackbarPos('HUE min' , 'HSV')
    h_max = cv2.getTrackbarPos('HUE max' , 'HSV')
    s_min = cv2.getTrackbarPos('SAT min' , 'HSV')
    s_max = cv2.getTrackbarPos('SAT max' , 'HSV')
    v_min = cv2.getTrackbarPos('VAL min' , 'HSV')
    v_max = cv2.getTrackbarPos('VAL max' , 'HSV')
#     print(h_min)
    lower = np.array([h_min ,s_min , v_min])
    upper = np.array([h_max ,s_max , v_max])
    mask =cv2.inRange(imghsv , lower ,upper)
    result  = cv2.bitwise_and(frame,frame ,mask =mask)
    cnts ,hei = cv2.findContours(mask , cv2.RETR_EXTERNAL,  cv2.CHAIN_APPROX_NONE)
    for i in cnts:
        area= cv2.contourArea(i)
        if area>300:
            peri =cv2.arcLength(i,True)
            approx =  cv2.approxPolyDP(i,0.02*peri ,True)
            x,y,w,h =cv2.boundingRect(i)
            cv2.rectangle(frame , (x,y) , (x+w ,y+h),(0,255,0) ,2)
            cv2.putText(frame , "points: "+str(len(approx)) , (x+w+10 , y+w+10) ,cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,255,0) , 2)

        if len(approx)==4:
            cv2.putText(frame , " Square "  , (x+w+10 , y+w+40) ,cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,255,0) , 2)
        elif len(approx)==3:
            cv2.putText(frame , "Triangle " , (x+w+10 , y+w+40) ,cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,255,0) , 2)
        else :
            cv2.putText(frame , "Circle "  , (x+w+10 , y+w+40) ,cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,255,0) , 2)
            
    
    cv2.imshow('mask' , mask)
    cv2.imshow('original' , frame)
    cv2.imshow('second' , result)
 
    
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
    
